In [1]:
import os
import argparse
from glob import glob
import numpy as np
from sklearn import metrics
import torch
from torch import nn
import torch.nn.functional as F

## utils

In [2]:
class Trainer:
    def __init__(self, model, dataset, args):
        self.device = args.device
        self.model = model.to(self.device)
        self.dataset = dataset
        self.epochs = args.num_epochs
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = args.learning_rate)
        self.criterion = nn.CrossEntropyLoss().to(self.device)
        self.val_idx = int(len(self.dataset) * (1-args.val_size))
        self.best_model_weights = self.model.state_dict()
        self.best_epoch = 0
        self.best_val_loss = float('inf')
        self.threshold = args.threshold
    def train(self):
        for epoch in range(1, self.epochs + 1):
            train_loss = 0.0
            val_loss = 0.0
            for i, data in enumerate(self.dataset):
                if i < self.val_idx:
                    loss = self._train_step(self.model, data)
                    train_loss += loss / self.val_idx
                else:
                    loss = self._val_step(self.model, data)
                    val_loss += loss / (len(self.dataset) - self.val_idx)
            # if epoch % 20 == 0 or epoch == self.epochs:
            #     print(f"""
            #         epoch {epoch}:
            #             train loss: {train_loss},
            #             val loss: {val_loss}
            #     """)
            if self.best_val_loss > val_loss:
                self.best_val_loss = val_loss
                self.best_epoch = epoch
                self.best_model_weights = model.state_dict()
        self.model.load_state_dict(self.best_model_weights)
        print(
            f"""
            best model loss is:
                val loss: {self.best_val_loss} @ epoch: {self.best_epoch}
            """
        )
        self._benchmark()
        return self.model

    def _train_step(self, model, data):
        self.optimizer.zero_grad()
        logits, target = self._shared_step(model, data)
        loss = self.criterion(logits, target)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    def _val_step(self, model, data):
        with torch.no_grad():
            logits, target = self._shared_step(model, data)
            loss = self.criterion(logits, target)
            return loss.item()
    def _shared_step(self, model, data):
        data.x = data.x.to(self.device)
        data.edge_index = data.edge_index.to(self.device)
        data.edge_attr = data.edge_attr.to(self.device)
        target = (data.y > self.threshold).long().to(self.device)
        logits = model(data)
        return logits, target
    def _benchmark(self):
        train_preds = []
        train_trues = []
        val_preds = []
        val_trues = []
        for i, data in enumerate(self.dataset):
            logits, target = self._shared_step(self.model, data)
            pred = logits.argmax(-1).cpu().numpy()
            target = target.cpu().numpy()
            if i < self.val_idx:
                train_preds.append(pred)
                train_trues.append(target)
            else:
                val_preds.append(pred)
                val_trues.append(target)
        train_preds = np.hstack(train_preds)
        train_trues = np.hstack(train_trues)
        val_preds = np.hstack(val_preds)
        val_trues = np.hstack(val_trues)
        print(
            f"""
                best model performance is:
                    train acc: {metrics.accuracy_score(train_trues, train_preds)}
                    val acc: {metrics.accuracy_score(val_trues, val_preds)}

                    train f1 score {metrics.f1_score(train_trues, train_preds)}
                    val f1 score {metrics.f1_score(val_trues, val_preds)}

                    train precision score {metrics.precision_score(train_trues, train_preds)}
                    val precision score {metrics.precision_score(val_trues, val_preds)}

                    train recall score {metrics.recall_score(train_trues, train_preds)}
                    val recall score {metrics.recall_score(val_trues, val_preds)}

                    num of pos prediction in training set {train_preds[train_preds == 1].shape[0]}
                    num of neg prediction in training set {train_preds[train_preds == 0].shape[0]}
                    num of pos prediction in val set {val_preds[val_preds == 1].shape[0]}
                    num of neg prediction in val set {val_preds[val_preds == 0].shape[0]}
            """
        )
        print(
            metrics.classification_report(val_trues, val_preds)
            )

## datasets

In [3]:
class sequentialDataset(torch.utils.data.Dataset):
    def __init__(self, fp, file_type = ".pt") -> None:
        self.data_files = sorted(glob(os.path.join(fp, f"*{file_type}")))
        super().__init__()
    def __getitem__(self, idx):
        return torch.load(self.data_files[idx])[0]
    def __getindex__(self, idx):
        return torch.load(self.data_files[idx])[0]
    def __len__(self):
        return len(self.data_files)

## model definition

In [4]:
from torch_geometric.nn import GCNConv
class GNN(torch.nn.Module):
    def __init__(self, input_size, feature_size, output_size):
        super().__init__()
        self.conv = GCNConv(in_channels=input_size, out_channels=feature_size)
        self.activation = nn.ReLU()
        self.fc = nn.Linear(in_features=feature_size, out_features=output_size)
    
    def forward(self, data):
        node_attr = F.normalize(data.x.float(), dim=0)
        num_pad = self.conv.in_channels - node_attr.shape[1]
        node_attr = torch.cat(
            (
                node_attr,
                torch.zeros((node_attr.shape[0], num_pad))
            ),
            -1
        )
        edge_index = data.edge_index.long()
        edge_weight = F.normalize(data.edge_attr.float().reshape(-1, 1), dim=0)
        x = self.conv(
                    x=node_attr, 
                    edge_index=edge_index, 
                    edge_weight=edge_weight
                    )
        x = self.activation(x)
        x = self.fc(x)
        return torch.sigmoid(x)

## training

In [5]:
graph_path = "../data/processed/news"
dataset = sequentialDataset(graph_path)
model = GNN(70, 32, 2)
args = dict(
    num_epochs = 500,
    learning_rate = 2e-4,
    device = "cpu",
    val_size = .2,
    threshold = 0
)

args = argparse.Namespace(**args)

In [6]:
trainer = Trainer(model, dataset, args)

In [7]:
model = trainer.train()


            best model loss is:
                val loss: 0.6515238881111145 @ epoch: 78
            

                best model performance is:
                    train acc: 0.6551724137931034
                    val acc: 0.6379310344827587

                    train f1 score 0.7916666666666666
                    val f1 score 0.7789473684210527

                    train precision score 0.6551724137931034
                    val precision score 0.6379310344827587

                    train recall score 1.0
                    val recall score 1.0

                    num of pos prediction in training set 435
                    num of neg prediction in training set 0
                    num of pos prediction in val set 116
                    num of neg prediction in val set 0
            
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        42
           1       0.64      1.00      0.78        74

    accuracy                  

/Users/jiekie/opt/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jiekie/opt/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jiekie/opt/anaconda3/envs/apulu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta